# Concise Formulation
### This code will formulate the MIP using Gurobi in Python. This has been edited from the version found in `formulation.ipynb` to provide the most concise formulation possible, while preserving the functionality needed.

In [1]:
import pandas as pd
from shapely.geometry import Point
import geopandas as gp
from geopandas import GeoDataFrame
import time
import gurobipy as gp
import numpy as np

## Parameters

In [132]:
G = 10000000
H = 78.91
alpha = 0.075
hub= 0.6
lamb = 250 * 365
n = 168
p = 74
q = 8
K = 74
uncertainty = 50
inverse_foods = [1/41.3, 1/107.38, 1/37.9961, 1/330.4]
NUM_HUBS = 8

## Data

In [133]:
arcDistance = pd.read_csv("./arcDistance.csv").iloc[0:168,:]
demand = pd.read_csv("./demand.csv").iloc[0:168,:]
supply = pd.read_csv("./supplyT.csv").iloc[0:168,:]
weighting = pd.read_csv("./weighting.csv")
inverse = pd.read_csv("./inverse_populations.csv").iloc[0:168,:]

## Constraint Formulation

In [142]:
model = gp.Model()

# model = gp.read("out.mps")

X = model.addMVar((n, n, p), lb=0)
Y = model.addMVar((n, NUM_HUBS, p), lb=0)
Z = model.addMVar((NUM_HUBS, n, p), lb=0)
beta = model.addVar(lb=0, vtype=gp.GRB.INTEGER)

lasttime = time.time()

for i in range(n):
    for k in range(p):
        if i < 8:
            model.addConstr(( sum(Y[:,i,k]) == sum(Z[i,:,k]) ))
        model.addConstr((sum(X[i,:,k]) + sum(Y[i,:,k]) <= abs(0.80 * supply.iloc[i,k])))
        for j in range(4):
            model.addConstr(( G * (sum(X[:,i,k]) + sum(Z[:,i,k])) * np.array(weighting)[k,j] <= abs(1.2 * demand.iloc[i,j])))
            model.addConstr((np.array(inverse)[i,0] * (np.array(demand)[i,j] - G * (sum(X[:,i,k]) + sum(Z[:,i,k])) * np.array(weighting)[k,j]) * np.array(inverse_foods)[j] <= beta))
        
    if i % 25 == 0:
        laptime = round((time.time() - lasttime), 2)
        print(f"{i} -- {str(laptime)} seconds")
        model.write("out.mps")
        lasttime = time.time()

model.write("out.mps")

0 -- 0.55 seconds
25 -- 12.51 seconds
50 -- 12.02 seconds
75 -- 12.03 seconds
100 -- 12.05 seconds
125 -- 12.05 seconds
150 -- 12.08 seconds


156

## Objective

In [143]:
objective = 0 

for i in range(168):
    for l in range(168):
        objective += arcDistance.iloc[i,l] * sum(X[i,l,:])
    for l in range(8):
        objective += arcDistance.iloc[i,l+168] * sum(Y[i,l,:]) * hub
        objective += arcDistance.iloc[i,l+168] * sum(Z[l,i,:])
    for k in range(p):
        for j in range(4):
            objective += 10478.822133354273 * (np.array(demand)[i,j] - G * (sum(X[:,i,k]) + sum(Z[:,i,k])) * np.array(weighting)[k,j])

# objective = objective * alpha * H
# objective += (1-alpha) * beta * lamb

## Optimization

In [144]:
# model = gp.read("out.mps")
model.setObjective(objective, gp.GRB.MINIMIZE)
model.optimize()

Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (linux64)
Thread count: 28 physical cores, 56 logical processors, using up to 28 threads
Optimize a model with 112480 rows, 2287489 columns and 13790784 nonzeros
Model fingerprint: 0xe13e8982
Variable types: 2287488 continuous, 1 integer (0 binary)
Coefficient statistics:
  Matrix range     [7e-06, 1e+09]
  Objective range  [2e+02, 2e+13]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e-01, 2e+14]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1.118246e+21
Presolve removed 78539 rows and 627905 columns (presolve time = 5s) ...
Presolve removed 89406 rows and 1016074 columns (presolve time = 10s) ...
Presolve removed 94265 rows and 1124517 columns (presolve time = 15s) ...
Presolve removed 108797 rows and 1994865 columns (presolve time = 46s) ...
Presolve removed 108797 rows and 1994865 columns (presolve time = 50s) ...
Presolve removed 

## Results

In [145]:
np.sum(np.sum(X.X))

4917536.722139654

In [146]:
np.sum(np.sum(Y.X))

278147.59148128016

In [147]:
np.sum(np.sum(Z.X))

278147.5914812801

In [148]:
beta.X

456.0

In [149]:
Xval = X.X
Yval = Y.X
Zval = Z.X
betaVal = beta.X

Xval = np.array(Xval)
Yval = np.array(Yval)
Zval = np.array(Zval)
betaVal = np.array(betaVal)

np.save('Xval_2.npy', Xval)
np.save('Yval_2.npy', Yval)
np.save('Zval_2.npy', Zval)
np.save('betaVal_2.npy', betaVal)
